# Info
Name:  

    ARM_SaSza_spetra_test

Purpose:  

    First go through of SaS-ze spectral data for testing application to cloud properties
  
Input:

    none at command line
  
Output:

    figures and save files...
  
Keywords:

    none
  
Dependencies:

    - load_utils.py : for loading netcdf files
    - matplotlib
    - numpy
    - scipy : for saving and reading
    - os
  
Needed Files:

  - file.rc : for consistent creation of look of matplotlib figures
  - ...
  
Modification History:

    Written: Samuel LeBlanc, NOAA-Boulder, 2019-02-26
    Modified: 

# Load the python modules and environment

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import load_utils as lu
from path_utils import getpath

In [31]:
import os
import Sp_parameters as Sp

In [2]:
%matplotlib notebook

In [7]:
fp = getpath('LASIC',make_path=True)
fp

u'/mnt/c/Users/sleblanc/Research/LASIC/'

In [8]:
daystr = '20160815'

# Load the files

In [10]:
fl = os.listdir(fp+'data/')

In [22]:
for g in fl: 
    if daystr in g:
        if 'vis' in g:
            glv = g
        if 'nir' in g:
            gln = g

In [23]:
glv,gln

(u'asisaszevisM1.a1.20160815.054702.cdf',
 u'asisaszenirM1.a1.20160815.054702.cdf')

In [24]:
vis,vish = lu.load_netcdf(fp+'data/'+glv,everything=True)

Reading file: /mnt/c/Users/sleblanc/Research/LASIC/data/asisaszevisM1.a1.20160815.054702.cdf
Outputting the Data subdatasets:
0: base_time
1: time_offset
2: time
3: wavelength
4: mio_temperature_mems
5: mio_temperature_mems_fahr
6: mio_temperature_trh
7: mio_rh
8: collector_temperature
9: collector_rh
10: collector_dewpoint
11: chiller_temperature
12: chiller_rh
13: chiller_dewpoint
14: collector_x_tilt
15: collector_y_tilt
16: collector_x_tilt_std
17: collector_y_tilt_std
18: band_azimuth
19: solar_azimuth
20: solar_zenith
21: inner_band_angle
22: inner_band_scattering_angle
23: clock_ticks
24: bench_temperature
25: ad_temperature
26: integration_time
27: number_of_scans
28: responsivity
29: zenith_radiance
30: lat
31: lon
32: alt
['chiller_rh', 'collector_y_tilt_std', 'base_time', 'mio_rh', 'mio_temperature_mems_fahr', 'number_of_scans', 'solar_zenith', 'zenith_radiance', 'wavelength', 'alt', 'ad_temperature', 'mio_temperature_mems', 'lon', 'chiller_dewpoint', 'bench_temperature', 'i

In [25]:
nir,nirh = lu.load_netcdf(fp+'data/'+gln,everything=True)

Reading file: /mnt/c/Users/sleblanc/Research/LASIC/data/asisaszenirM1.a1.20160815.054702.cdf
Outputting the Data subdatasets:
0: base_time
1: time_offset
2: time
3: wavelength
4: mio_temperature_mems
5: mio_temperature_mems_fahr
6: mio_temperature_trh
7: mio_rh
8: collector_temperature
9: collector_rh
10: collector_dewpoint
11: chiller_temperature
12: chiller_rh
13: chiller_dewpoint
14: collector_x_tilt
15: collector_y_tilt
16: collector_x_tilt_std
17: collector_y_tilt_std
18: band_azimuth
19: solar_azimuth
20: solar_zenith
21: inner_band_angle
22: inner_band_scattering_angle
23: clock_ticks
24: bench_temperature
25: ad_temperature
26: integration_time
27: number_of_scans
28: responsivity
29: zenith_radiance
30: lat
31: lon
32: alt
['chiller_rh', 'collector_y_tilt_std', 'base_time', 'mio_rh', 'mio_temperature_mems_fahr', 'number_of_scans', 'solar_zenith', 'zenith_radiance', 'wavelength', 'alt', 'ad_temperature', 'mio_temperature_mems', 'lon', 'chiller_dewpoint', 'bench_temperature', 'i

In [29]:
vish['zenith_radiance'], vish['wavelength']

(<type 'netCDF4._netCDF4.Variable'>
 float32 zenith_radiance(time, wavelength)
     long_name: Spectral zenith radiance from Si VIS spectrometer
     units: W/(m^2 um sr)
     valid_min: 0.0
     missing_value: -9999.0
 unlimited dimensions: time
 current shape = (39258, 2048)
 filling off, <type 'netCDF4._netCDF4.Variable'>
 float32 wavelength(wavelength)
     long_name: Wavelength of VIS spectrometer pixels
     units: nm
 unlimited dimensions: 
 current shape = (2048,)
 filling off)

In [30]:
nirh['zenith_radiance'], nirh['wavelength']

(<type 'netCDF4._netCDF4.Variable'>
 float32 zenith_radiance(time, wavelength)
     long_name: Spectral zenith radiance from InGaAs NIR spectrometer
     units: W/(m^2 um sr)
     valid_min: 0.0
     missing_value: -9999.0
 unlimited dimensions: time
 current shape = (39258, 256)
 filling off, <type 'netCDF4._netCDF4.Variable'>
 float32 wavelength(wavelength)
     long_name: Wavelength of NIR spectrometer pixels
     units: nm
 unlimited dimensions: 
 current shape = (256,)
 filling off)

# Plot out some data

In [32]:
help(Sp.Sp)

Help on class Sp in module Sp_parameters:

class Sp
 |  Purpose:
 |    Sp: spectrum class object that has all the tools for easy spectra analysis.
 |    Includes tools for building look up tables
 |  
 |  keywords:
 |    - irrad: (default False) if True, then calculates the irradiance values as well
 |    - verbose (default True) if True, then returns many comments while processing
 |    - liq_only (default False) if True, then only computes the liquid side of the lut
 |    - ice_only (default False) if True only computes the ice lut
 |  
 |  
 |  Modification History:
 |  Modified (v1.1): Samuel LeBlanc, NASA Ames, 2015-05-14
 |                  - added in _init_ for checking if using radiance subset instead of all radiances 
 |                    (rad vs. rads), in matlab loaded values
 |  Modified (v1.2): Samuel LeBlanc, NASA Ames, 2015-10-01
 |                  - added subset of wavelengths for normalization
 |  Modified (v1.3): Samuel LeBlanc, NASA Ames, 2015-11-02
 |             